# Convert the wav files into spectrograms

In [ ]:
# from matplotlib.colors import Normalize
import numpy as np
from copy import deepcopy

import scipy.io.wavfile
from scipy.io.wavfile import read
from scipy import signal

In [ ]:
# Directory of where to find the wav files
rootdir = 'wav'

output_x_name = 'data_x_spectrum'
output_y_name = 'data_y_spectrum'

In [ ]:
# x, sr = librosa.load(filename, sr=None, mono=True)
# mel = librosa.feature.melspectrogram(y=x, sr=sr, hop_length=hop_length)
# log_mel = np.transpose(librosa.logamplitude(mel))

In [ ]:
MAX_SONGS = 8000

numSongs = 0


dim = 1000 # length of the samples
offset = 2000 # start getting song values at this time

data_x = np.zeros((MAX_SONGS, 129, dim))
data_y = []

for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        try:
            if not file.endswith(".wav"):
                continue
            if (numSongs %100 == 0):
                print(numSongs, ' songs processed')
                
            track_id = int(os.path.splitext(file)[0])

            sr_value, x_value = scipy.io.wavfile.read(os.path.join(subdir,file))
            f, t, Sxx = signal.spectrogram(x=x_value,fs=sr_value, axis=0)
            
            Sxx[Sxx<0]= 0
            Sxx+=1e-8
            Sxx = np.log(Sxx)
            
            if Sxx.shape[1] < offset+dim:
                continue
                
            numSongs += 1
            if (numSongs > MAX_SONGS):
                break
                
            data_x[numSongs-1, :, :] = Sxx[:,offset:offset+dim]
            data_y.append(genre[genre['track_id'] == track_id]['genre_top'].iloc[0])
        
        except Exception as e: 
            print('Error with this file', e)
    if (numSongs > MAX_SONGS):
        break
        
# in case numSongs < max_songs
data_x = data_x[0:numSongs-1, :, :]

data_y = np.array(data_y)
data_y = data_y[0:numSongs-1]

np.save(output_x_name, data_x)
np.save(output_y_name, data_y)

In [ ]:

fig = plt.figure(figsize=(20,5))
plt.pcolormesh(t, f, Sxx)
plt.savefig('spectrum')
plt.show()